In [ ]:
#@title Check GPU
!nvidia-smi

Wed Nov 16 18:15:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    48W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets --q

     |████████████████████████████████| 451 kB 7.7 MB/s 
     |████████████████████████████████| 115 kB 86.5 MB/s 
     |████████████████████████████████| 212 kB 92.2 MB/s 
     |████████████████████████████████| 182 kB 86.9 MB/s 
     |████████████████████████████████| 127 kB 84.7 MB/s 


In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 5.5 MB 8.3 MB/s 
     |████████████████████████████████| 7.6 MB 76.8 MB/s 


In [ ]:
#@title Imports
import nltk
import numpy as np
import tensorflow as tf
import pandas as pd
import transformers

from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from datasets import list_datasets, load_dataset
from pprint import pprint
from transformers import BertTokenizer, TFBertModel

from sklearn.model_selection import train_test_split


from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [ ]:
import torch


In [ ]:
#@title Load Dataset
imdb_train = load_dataset('imdb', split = 'train')
imdb_test = load_dataset('imdb', split = 'test')

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


#### Pegasus

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

model = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

Downloading:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

Some layers of TFPegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

In [ ]:
model.summary()

Model: "tf_pegasus_for_conditional_generation"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
 final_logits_bias (BiasLaye  multiple                 96103     
 r)                                                              
                                                                 
Total params: 569,844,583
Trainable params: 569,748,480
Non-trainable params: 96,103
_________________________________________________________________


##### Tests

In [ ]:
imdb_train[0]['text']

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
imdb_train[0]['label']

0

In [ ]:
#test summary generation
inputs = tokenizer(imdb_train[0]['text'], max_length=200, truncation=True, return_tensors="tf")


In [ ]:
inputs

{'input_ids': <tf.Tensor: shape=(1, 200), dtype=int32, numpy=
array([[  125, 11853,   125,  3887,   110, 43669, 42566,   121,  2916,
        40979,  1629,   135,   161,   545,   794,   262,   113,   149,
          109, 11678,   120,  4892,   126,   173,   126,   140,   211,
         1291,   115, 42366,   125,   163,  1455,   120,   134,   211,
          126,   140, 15924,   141,   475,   107,   283,   107, 10498,
          175,   126,   521,  1424,   112,  1646,   136,   531,   108,
         1923,   270,   114,  1781,   113,  3265,  1341,   198, 88675,
          194,   125,   288,   196,   112,   236,   136,   118,  1150,
          107,   105, 10226, 48003,   105, 10226, 48003,   159,  4715,
          117, 10609,   279,   114,   758,  8518,  4970,   980,  1729,
        32777,   170,  1728,   112,   543,   579,   265,   137,   160,
          271,   107,   222,   970,   265,  1728,   112,   777,   215,
         1090,   116,   112,   395,   181,  1851,   113,  6387,   124,
          180, 

In [ ]:
summary_ids = model.generate(inputs["input_ids"], 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=10,
                              max_length=20)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

'I AM CURIOUS is a film that has been described as "pornography on steroids'


In [ ]:
test = []
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
test.append(summary)
test.append('test words here')
print(test)

In [ ]:
def generate_summaries(input_text):
  inputs = tokenizer(input_text, max_length=200, truncation=True, return_tensors="tf")
  summary_ids = model.generate(inputs["input_ids"], 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=20,
                              max_length=50)
  summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  return summary

In [ ]:
test_array = ['this','is','a','test']
np.savetxt("test_save.txt", test_array, fmt = '%s')

In [ ]:
print(generate_summaries(imdb_train[2]['text']))

A film about a tree growing in the middle of nowhere, with no real purpose or meaning.


In [ ]:
batch_1 = imdb_train[:100]
batch_2 = imdb_train[100:200]
batch_3 = imdb_train[200:300]
batch_4 = imdb_train[300:400]
batch_5 = imdb_train[400:500]

In [ ]:
batch_1['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [ ]:
imdb_train_summaries_1 = []
i = 0
for item in batch_1['text']:
  summary = generate_summaries(item)
  imdb_train_summaries_1.append(summary)
  i += 1
  if i%5 == 0:
    print(i)
np.savetxt("imdb_summaries_1.txt", imdb_train_summaries_1, fmt = '%s')

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [ ]:
imdb_train_summaries_2 = []
i = 0
for item in batch_2['text']:
  summary = generate_summaries(item)
  imdb_train_summaries_2.append(summary)
  i += 1
  if i%5 == 0:
    print(i)
np.savetxt("imdb_summaries_4.txt", imdb_train_summaries_2, fmt = '%s')

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [ ]:
imdb_train_summaries_3 = []
i = 0
for item in batch_3['text']:
  summary = generate_summaries(item)
  imdb_train_summaries_3.append(summary)
  i += 1
  if i%5 == 0:
    print(i)
np.savetxt("imdb_summaries_3.txt", imdb_train_summaries_3, fmt = '%s')

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [ ]:
imdb_train_summaries_4 = []
i = 0
for item in batch_4['text']:
  summary = generate_summaries(item)
  imdb_train_summaries_4.append(summary)
  i += 1
  if i%5 == 0:
    print(i)
np.savetxt("imdb_summaries_2.txt", imdb_train_summaries_4, fmt = '%s')

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [ ]:
imdb_train_summaries_5 = []
i = 0
for item in batch_5['text']:
  summary = generate_summaries(item)
  imdb_train_summaries_5.append(summary)
  i += 1
  if i%5 == 0:
    print(i)
np.savetxt("imdb_summaries_5.txt", imdb_train_summaries_5, fmt = '%s')

5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


##### Batch Summarization

In [ ]:
imdb_train[24998]['text']

'\'The Adventures Of Barry McKenzie\' started life as a satirical comic strip in \'Private Eye\', written by Barry Humphries and based on an idea by Peter Cook. McKenzie ( \'Bazza\' to his friends ) is a lanky, loud, hat-wearing Australian whose two main interests in life are sex ( despite never having had any ) and Fosters lager. In 1972, he found his way to the big screen for the first of two outings. It must have been tempting for Humphries to cast himself as \'Bazza\', but he wisely left the job to Barry Crocker ( later to sing the theme to the television soap opera \'Neighbours\'! ). Humphries instead played multiple roles in true Peter Sellers fashion, most notably Bazza\'s overbearing Aunt \'Edna Everage\' ( this was before she became a Dame ).<br /><br />You know this is not going to be \'The Importance Of Being Ernest\' when its censorship classification N.P.A. stands for \'No Poofters Allowed\'. Pom-hating Bazza is told by a Sydney solicitor that in order to inherit a share i

In [ ]:
s = 114
i = 22600
while 25000/i != 1:
  #status check on iteration number
  print('s= ' + str(s))
  print('i = ' + str(i))

  end = i + 200
  #tokenize in batch of 200
  inputs = tokenizer(imdb_train[i:end]['text'], max_length = 300, truncation=True, return_tensors="tf", padding='longest')
  
  #generate summary from tokens
  summary_ids = model.generate(inputs["input_ids"], 
                                num_beams=1,
                                no_repeat_ngram_size=1,
                                min_length=20,
                                max_length=100)

  #decode to text
  imdb_summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

  #create df
  df = pd.DataFrame(imdb_summary, columns = ['summary'])

  #write to file
  path_name = '/content/gdrive/MyDrive/imdb_summary_' + str(s) +'.csv'
  df.to_csv(path_name)
  print('file saved: ' + path_name)

  #iteration counter update
  i += 200
  s += 1

In [ ]:
s = 114
i = 22600
while 25000/i != 1:
  #status check on iteration number
  print('s= ' + str(s))
  print('i = ' + str(i))

  end = i + 100
  #tokenize in batch of 100
  inputs = tokenizer(imdb_train[i:end]['text'], max_length = 300, truncation=True, return_tensors="tf", padding='longest')
  
  #generate summary from tokens
  summary_ids = model.generate(inputs["input_ids"], 
                                num_beams=1,
                                no_repeat_ngram_size=1,
                                min_length=20,
                                max_length=100)

  #decode to text
  imdb_summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

  #create df
  df = pd.DataFrame(imdb_summary, columns = ['summary'])

  #write to file
  path_name = '/content/gdrive/MyDrive/imdb_summary_' + str(s) +'.csv'
  df.to_csv(path_name)
  print('file saved: ' + path_name)

  #iteration counter update
  i += 100
  s += 1

s= 114
i = 22600
file saved: /content/gdrive/MyDrive/imdb_summary_114.csv
s= 115
i = 22700
file saved: /content/gdrive/MyDrive/imdb_summary_115.csv
s= 116
i = 22800
file saved: /content/gdrive/MyDrive/imdb_summary_116.csv
s= 117
i = 22900
file saved: /content/gdrive/MyDrive/imdb_summary_117.csv
s= 118
i = 23000
file saved: /content/gdrive/MyDrive/imdb_summary_118.csv
s= 119
i = 23100
file saved: /content/gdrive/MyDrive/imdb_summary_119.csv
s= 120
i = 23200
file saved: /content/gdrive/MyDrive/imdb_summary_120.csv
s= 121
i = 23300
file saved: /content/gdrive/MyDrive/imdb_summary_121.csv
s= 122
i = 23400
file saved: /content/gdrive/MyDrive/imdb_summary_122.csv
s= 123
i = 23500
file saved: /content/gdrive/MyDrive/imdb_summary_123.csv
s= 124
i = 23600
file saved: /content/gdrive/MyDrive/imdb_summary_124.csv
s= 125
i = 23700
file saved: /content/gdrive/MyDrive/imdb_summary_125.csv
s= 126
i = 23800
file saved: /content/gdrive/MyDrive/imdb_summary_126.csv
s= 127
i = 23900
file saved: /content/

In [ ]:
#@title Batch 4 beam 500 max length
s = 1
i = 0
nz = 1
while 25000/(i+nz) != 1:
  #status check on iteration number
  print('s= ' + str(s))
  print('i = ' + str(i))

  end = i + 10
  #tokenize in batch of 100
  inputs = tokenizer(imdb_train[i:end]['text'], max_length = 400, truncation=True, return_tensors="tf", padding='longest')
  
  #generate summary from tokens
  summary_ids = model.generate(inputs["input_ids"], 
                                num_beams=3,
                                no_repeat_ngram_size=1,
                                min_length=50,
                                max_length=400)

  #decode to text
  imdb_summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

  #create df
  df = pd.DataFrame(imdb_summary, columns = ['summary'])

  #write to file
  path_name = '/content/gdrive/MyDrive/imdb_pegasus_4_beam_summaries/imdb_summary_' + str(s) +'.csv'
  df.to_csv(path_name)
  print('file saved: ' + path_name)

  #iteration counter update
  i += 10
  s += 1
  if i > 0:
    nz = 0

s= 1
i = 0
file saved: /content/gdrive/MyDrive/imdb_pegasus_4_beam_summaries/imdb_summary_1.csv
s= 2
i = 10
file saved: /content/gdrive/MyDrive/imdb_pegasus_4_beam_summaries/imdb_summary_2.csv
s= 3
i = 20


In [ ]:
inputs = tokenizer(imdb_train[:200]['text'], max_length = 300, truncation=True, return_tensors="tf", padding='longest')


In [ ]:
inputs

{'input_ids': <tf.Tensor: shape=(10, 500), dtype=int32, numpy=
array([[  125, 11853,   125, ...,     0,     0,     0],
       [  198,   187,  5265, ...,     0,     0,     0],
       [  240,   209,   112, ...,     0,     0,     0],
       ...,
       [  434,   125,   211, ...,     0,     0,     0],
       [ 2632,   127,   219, ...,   124,   169,     1],
       [  182,   117,   243, ...,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(10, 500), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>}

In [ ]:
summary_ids = model.generate(inputs["input_ids"], 
                              num_beams=1,
                              no_repeat_ngram_size=1,
                              min_length=20,
                              max_length=100)
# pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

In [ ]:
imdb_summary_1 = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)

In [ ]:
df = pd.DataFrame(imdb_summary_1, columns = ['summary'])

In [ ]:
df

,summary
0,I AM CURIOUS is a film that has been viewed by...
1,"Chloe Rigny's new film is a ""damning indictmen..."
2,A film about a tree growing in the middle of n...
3,"The film is a farce, but it's not as bad or st..."
4,"I'm not sure what happened to the film, but it..."
...,...
195,This is a film that should not be watched by a...
196,"This is a review of the film, which I did not ..."
197,"The Klondike, the new movie about Canada's ear..."
198,The greatest western ever made is being releas...


In [ ]:
  path_name = '/content/gdrive/MyDrive/imdb_summary_1.csv'
  df.to_csv(path_name)

In [ ]:
print('file saved: ' + path_name)

file saved: /content/gdrive/MyDrive/imdb_summary_1.csv


In [ ]:
pprint(imdb_train[:50]['text'])

['I rented I AM CURIOUS-YELLOW from my video store because of all the '
 'controversy that surrounded it when it was first released in 1967. I also '
 'heard that at first it was seized by U.S. customs if it ever tried to enter '
 'this country, therefore being a fan of films considered "controversial" I '
 'really had to see this for myself.<br /><br />The plot is centered around a '
 'young Swedish drama student named Lena who wants to learn everything she can '
 'about life. In particular she wants to focus her attentions to making some '
 'sort of documentary on what the average Swede thought about certain '
 'political issues such as the Vietnam War and race issues in the United '
 'States. In between asking politicians and ordinary denizens of Stockholm '
 'about their opinions on politics, she has sex with her drama teacher, '
 'classmates, and married men.<br /><br />What kills me about I AM '
 'CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. '
 "Really, 

In [ ]:
tokenizer.batch_decode(summary_ids)

['I AM CURIOUS-YELLOW is a film that has been considered by many to be one of the most controversial in Swedish cinema.',
 'Chloe Rigny\'s new film is a "damning indictment" of the US culture that treats women as second-class citizens when it comes to nudity.',
 'A film about a tree growing in the middle of nowhere, with no real purpose or meaning.',
 "The film is a farce, but it's not as bad or stupid and I think the director has done an excellent job.",
 "I'm not sure what the hell happened to that film, but it was a hoot.",
 "The film is a disgrace to the cinema. ⁇ br /> Filmed in black and white, it's an amateur photographer who takes pictures of his rabbit on school night!",
 'I\'m writing to express my dismay at the inaccuracies in "I Love Lucy", which is a great film but has so many mistakes that it cannot be called an accurate portrayal of her life.',
 '"Before Laughter" is a movie about the life of comedian Lucille Ball and her husband Desi Arnaz.',
 'Peter Bogdanovich\'s late

In [ ]:
num_train_examples = 20000
num_val_examples = 5000
max_length = 512

x_summary_train = bert_tokenizer(imdb_train_summaries[:num_train_examples],
                                 max_length = max_length,
                                 truncation = True,
                                 padding = 'max_length',
                                 return_tensors='tf')
y_train = imdb_train[:num_train_examples]['label']

x_summary_val = bert_tokenizer(imdb_train_summaries[num_train_examples:], 
                               max_length = max_length,
                               truncation = True,
                               padding = 'max_length',
                               return_tensors='tf')
y_val = imdb_train[num_train_examples:]['label']

In [ ]:
bert_summary_train_input_ids, bert_train_labels = select_min_length_examples(x_summary_train, y_train)
bert_summary_val_input_ids, bert_val_labels = select_min_length_examples(x_summary_val, y_val)

In [ ]:
bert_classification_model = create_bert_cls_model()
bert_classification_model.summary()
# keras.utils.plot_model(bert_classification_model, show_shapes=True, dpi=90)

bert_classification_model_history = bert_classification_model.fit(
    bert_summary_train_input_ids,
    bert_train_labels,
    validation_data=(bert_summary_val_input_ids, bert_val_labels),
    batch_size=8,
    epochs=2
)

### Pegasus Pretrained on Amazon Reviews

In [ ]:
!ls 

gdrive	sample_data


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_ckpt = 'gdrive/MyDrive/model_weights/pegasus-amazon2'
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

In [ ]:
def create_summary(dataset, model, tokenizer, 
                   batch_size=16, device=device, 
                   column_text="text"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    
    all_summaries = []
    
    for article_batch in article_batches:
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        
        #decode summary
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                              clean_up_tokenization_spaces=True) 
        for s in summaries]

        all_summaries.append(decoded_summaries)
    
    
    return all_summaries

In [ ]:
imdb_train

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

#### Create Summaries

In [ ]:
#@title Summary Creation
summary_test = create_summary(imdb_train, model_pegasus, tokenizer)

In [ ]:
all_summaries = list(chain(*summary_test))

In [ ]:
  df = pd.DataFrame(all_summaries, columns = ['summary'])
  path_name = '/content/gdrive/MyDrive/imdb_amazon_summary.csv'
  df.to_csv(path_name)

### Test Section

In [ ]:
#@title test summaries
summary_test = create_summary(imdb_train[:32], model_pegasus, tokenizer)

In [ ]:
len(summary_test)

2

In [ ]:
from itertools import chain
test_variable = list(chain(*summary_test))

In [ ]:
len(test_variable)

32

In [ ]:
test_variable[0]

"I AM CURIOUS-YELLOW is a good film for anyone wanting to study the meat and potatoes (no pun intended) of Swedish cinema. But really, this film doesn't have much of a plot."

In [ ]:
summary_test[1]

['Too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots',
 "It's so bad I can't even watch it, and it's so bad I can't believe I've seen it. It's so bad I can't believe I've seen it, and it's so bad I can't believe I've seen it.",
 "I think I will make a movie next weekend. Oh wait, I'm working..oh I'm sure I can fit it in. It looks like whoever made this film fit it in. I hope the makers of this crap have day jobs because this film sucked!!!",
 "TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen 

In [ ]:
imdb_train[16]

{'text': "My interest in Dorothy Stratten caused me to purchase this video. Although it had great actors/actresses, there were just too many subplots going on to retain interest. Plus it just wasn't that interesting. Dialogue was stiff and confusing and the story just flipped around too much to be believable. I was pretty disappointed in what I believe was one of Audrey Hepburn's last movies. I'll always love John Ritter best in slapstick. He was just too pathetic here.",
 'label': 0}

In [ ]:
len(summary_test)

1

In [ ]:
summary_test

['Too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots, too many subplots',
 "It's so bad I can't even watch it, and it's so bad I can't believe I've seen it. It's so bad I can't believe I've seen it, and it's so bad I can't believe I've seen it.",
 "I think I will make a movie next weekend. Oh wait, I'm working..oh I'm sure I can fit it in. It looks like whoever made this film fit it in. I hope the makers of this crap have day jobs because this film sucked!!!",
 "TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen in a long time. TZC is the worst movie I've seen 

In [ ]:
#@title test summaries
summary_test2 = create_summary(imdb_train[0], model_pegasus, tokenizer)

In [ ]:
summary_test2

['A plot that made me want to watch it again and again and again and again and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over and over']

In [ ]:
#@title Processes Check
!nvidia-smi

Wed Nov 16 17:56:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |  15082MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------